In [6]:
#데이터 셋
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [2]:
# 404 훈련 샘플 13개의 특성
train_data.shape

(404, 13)

In [3]:
test_data.shape

(102, 13)

In [5]:
# 주택 가격
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

In [8]:
# 데이터 전처리
# 훈련데이터에서 계산한 값을 사용하여 정규화
# 테스트 데이터나 새로운 데이터도 훈련 데이터에서 사용한 계산 값을 사용해야함

mean = train_data.mean(axis=0)
train_data -= mean

std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std


In [10]:
# 모델 정의
# 훈련 데이터 수가 적을 수록 과대적합이 쉽게 일어나서 작은 모델을 사용하는 것을 추천
from keras import models
from keras import layers

# 동일한 모델을 여러번 생성할 것이므로 함수를 만들어 사용
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1], )))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1)) # 하나의 유닛에 활서화 함수 없음, 전형적인 스칼라 회귀, 순수한 선형 층, 어떤 범위의 값이라도 예측하도록 냅둠
    model.compile(optimizer='rmsprop', loss='mse',metrics=['mae'])
    return model

In [13]:
# 훈련 검증
# K-겹 검증

import numpy as np

k=4

num_val_samples = len(train_data) // k
print(num_val_samples)
num_epochs = 100
all_scores = []

for i in range(k):
    print('처리중인 폴드 #', i)
    
    # 검증 데이터 준비 : K 번째 분할
    val_data    =    train_data[ i * num_val_samples : (i+1) * num_val_samples]
    val_targets = train_targets[ i * num_val_samples : (i+1) * num_val_samples]
    
    # 훈련 데이터 준비 : 다른 분할 전체
    partial_train_data    = np.concatenate( [   train_data[ : i*num_val_samples],    train_data[(i+1)*num_val_samples : ]], axis=0)
    partial_train_targets = np.concatenate( [train_targets[ : i*num_val_samples], train_targets[(i+1)*num_val_samples : ]], axis=0)
    
    # 모델 구성 및 컴파일
    model = build_model()
    
    # 모델 훈련
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    
    # 모델 평가 : 검증 세트
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    
    all_scores.append(val_mae)
    

101
처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3


In [14]:
# 각 폴드별 검증 점수
all_scores

[2.525575637817383, 2.4528229236602783, 2.5700507164001465, 2.5373544692993164]

In [16]:
# 최종 평균 점수
np.mean(all_scores)

2.521450936794281

In [18]:
# 최종 모델 훈련하기
# 매개변수 튜닝 후 새롭게 컴파일 된 모델
model = build_model()

# 전체 데이터로 훈련
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)
    
# 모델 평가 : 검증 세트
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

print('\n')
print(test_mse_score)
print('\n')
print(test_mae_score)

4/4 [==============================] - 0s 1ms/step - loss: 922.7816 - mae: 29.5433


922.7815551757812


29.543319702148438
